# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [49]:
#%pylab inline
%matplotlib inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

In [50]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [51]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [52]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(0, len(classes)):
    data, target = load_pose_data(i)
    all_data += data
    all_target += target

all_data = np.asarray(all_data)
all_target = np.asarray(all_target)
print('total number of data:', len(all_data))

ValueError: could not convert string to float

In [40]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [41]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [42]:
# YOUR CODE HERE
clf.fit(all_data[training_data], all_target[training_data])

TypeError: only integer scalar arrays can be converted to a scalar index

### predicting

In [43]:
clf.predict(all_data[training_data]), all_target[training_data]

TypeError: only integer scalar arrays can be converted to a scalar index

In [44]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [45]:
expected = []
predicted = []
# YOUR CODE HERE
for i in range(len(all_data)):
    if i not in training_data:
        expected.append(all_target[i])
        predicted.append(clf.predict([all_data[i]])[0])

evaluate(expected, predicted)

ValueError: max() iterable argument is empty

## 4. Evaluate on the test data

In [53]:
expected = []
predicted = []
# YOUR CODE HERE
for i in range(len(all_data)):
    if i not in training_data:
        expected.append(all_target[i])
        predicted.append(clf.predict([all_data[i]])[0])
        
evaluate(expected, predicted)

ValueError: max() iterable argument is empty

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [54]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [55]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

TypeError: a bytes-like object is required, not 'str'